In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
#from konlpy.tag import mecab
from gensim.models import Word2Vec
from scipy.spatial import distance
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer

## 전처리

In [2]:
raw_df = pd.read_csv('../Data/data_for_use/theme_contents.csv', encoding = 'utf-8', index_col=0)
raw_df.dropna(inplace=True)

In [3]:
raw_df

,theme,contents
0,2차전지,신종 코로나바이러스 감염증(코로나19) 사태로 인한 급락장에서 10대 그룹 시가총...
1,2차전지,2차전지·디스플레이 공정 장비 전문기업 나인테크가 기업인수목적회사(SPAC·스팩) ...
2,2차전지,2차전지·디스플레이 공정 장비 전문기업 나인테크가 교보7호스팩과 합병을 통한 코스...
3,2차전지,'울산형 뉴딜사업' 추진 등 코로나19 이후 지역경제 살리기에 나선 울산에 반가운...
4,2차전지,코스닥 2차 전지 소재 기업 에코프로비엠이 압도적 기술력을 바탕으로 가파른 성장 ...
...,...,...
33595,희토류,희귀금속(희토류 등) 테마 주요 종목 등락률씽크풀 로보알고리즘 RASSI의 분석에...
33596,희토류,희귀금속(희토류 등) 테마 주요 종목 등락률씽크풀 로보알고리즘 RASSI의 분석에...
33597,희토류,희귀금속(희토류 등) 테마 주요 종목 등락률씽크풀 로보알고리즘 RASSI의 분석에...
33598,희토류,희귀금속(희토류 등) 테마 주요 종목 등락률씽크풀 로보알고리즘 RASSI의 분석에...


In [5]:
raw_df = raw_df[['theme', 'contents']]
raw_df['contents'] = raw_df['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
raw_df.head()

,theme,contents
0,2차전지,신종 코로나바이러스 감염증코로나 사태로 인한 급락장에서 대 그룹 시가총액 반등을 ...
1,2차전지,차전지디스플레이 공정 장비 전문기업 나인테크가 기업인수목적회사스팩 합병을 통해 코스...
2,2차전지,차전지디스플레이 공정 장비 전문기업 나인테크가 교보호스팩과 합병을 통한 코스닥 상...
3,2차전지,울산형 뉴딜사업 추진 등 코로나 이후 지역경제 살리기에 나선 울산에 반가운 소식이...
4,2차전지,코스닥 차 전지 소재 기업 에코프로비엠이 압도적 기술력을 바탕으로 가파른 성장 추...


In [6]:
tok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)

[██████████████████████████████████████████████████]


In [7]:
# tokenize data

raw_tokenized_data = []
for content in tqdm(raw_df['contents']):
    token = sp(content)
    stop_words = ['으로', '로도', '지만', '에서', '려는', '하다', '은', '는', '이', '가', '을', '를', '수', '할']
    #for word in token:
        #if len(word) == 1:
            #stop_words.append(word)
    final_tokens = [word for word in token if word not in stop_words]
    raw_tokenized_data.append(final_tokens)

100%|██████████| 33600/33600 [00:22<00:00, 1520.24it/s]


In [8]:
# make train_data

train_data = raw_df.copy()
train_data['tokens'] = pd.Series(raw_tokenized_data)

In [12]:
train_data.head()

33600

## Model Train

In [10]:
news_tokens = train_data['tokens'].tolist()  # data input as list

# params
v_dimension = 300
v_window = 8

model = Word2Vec(sentences = news_tokens, size = v_dimension, window = v_window, min_count = 5, workers = 4, sg = 0)

In [126]:
model.save('word2vec.model')

In [11]:
model.wv.vectors.shape

(7274, 300)

In [13]:
# sentence vectors without normalization

def without_normal(tokens):
    vectors = []
    for token in tokens:
        init_v = np.array([0.0]*v_dimension)
        for word in token:
            word_vectors = model.wv
            if word in word_vectors.vocab:
                v = model.wv[word]
                init_v = init_v + v
        vectors.append(init_v)
    
    frame = { 'themes': train_data['theme'].tolist(), 'vectors': vectors }
    result = pd.DataFrame(frame) 
    
    return result

In [14]:
# theme vectors without normalization

def theme_without_normal(news_df):
    theme_list = []
    vector_list = []
    for theme in news_df['themes'].unique():
        temp_df = news_df.loc[news_df['themes'] == theme]
        add_v = np.array([0.0]*v_dimension)
        for vec in temp_df['vectors']:
            add_v  = add_v + vec
        theme_list.append(theme)
        vector_list.append(add_v)
        
    frame = { 'themes': theme_list, 'vectors': vector_list }
    result = pd.DataFrame(frame)
    
    return result

In [15]:
news_vectors_df = without_normal(train_data['tokens'])

In [16]:
news_vectors_df.head()

,themes,vectors
0,2차전지,"[-144.25111735332757, -198.2817719373852, 108...."
1,2차전지,"[-101.9521975889802, -28.009637451847084, 83.0..."
2,2차전지,"[-106.2103034351021, -82.91742928058375, 108.5..."
3,2차전지,"[-216.55495595373213, -362.1210448469501, 99.8..."
4,2차전지,"[-347.2697988282889, -240.06703539006412, -4.9..."


In [22]:
theme_vectors_df = theme_without_normal(news_vectors_df)

In [23]:
theme_vectors_df.head()

,themes,vectors
0,2차전지,"[-12996.721097633927, -31164.081225086906, 884..."
1,3D프린터,"[-25000.227676852548, -26006.33596550407, 3427..."
2,4대강,"[-15137.951472495974, -35683.33157003227, 3616..."
3,5G,"[-1509.4786461966432, -26933.293039654833, 224..."
4,AR,"[-21946.258229792904, -24954.065020793714, 185..."


In [27]:
theme_vectors_df

,themes,vectors
0,2차전지,"[-12996.721097633927, -31164.081225086906, 884..."
1,3D프린터,"[-25000.227676852548, -26006.33596550407, 3427..."
2,4대강,"[-15137.951472495974, -35683.33157003227, 3616..."
3,5G,"[-1509.4786461966432, -26933.293039654833, 224..."
4,AR,"[-21946.258229792904, -24954.065020793714, 185..."
...,...,...
163,홈쇼핑,"[-27733.398158481534, -30374.903595016585, 977..."
164,화장품,"[-25628.57060019402, -24789.78141408258, 12859..."
165,화학섬유,"[-1160.0935827617286, -1765.1668276214623, 288..."
166,흑사병,"[-40173.6514197578, -2965.622421242573, 46262...."


In [17]:
theme_vectors_df.to_csv('theme_vectors_xnorm')

## Test

In [28]:
# news vectors without normalization

def vectorize_without_normal(news):
    news_words = news.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    token = sp(news)
    #stop_words = ['으로', '로도', '지만', '에서', '려는', '하다']
    #for word in token:
        #if len(word) == 1:
            #stop_words.append(word)
    final_tokens = [word for word in token if word not in stop_words]
    init_v = np.array([0.0]*v_dimension)
    for word in final_tokens:
        word_vectors = model.wv
        if word in word_vectors.vocab:
            v = model.wv[word]
            init_v = init_v + v
    return init_v

In [29]:
# test

def test(news):
    news_vec = vectorize_without_normal(news)
    result = []
    for theme in theme_vectors_df['vectors']:
        cosine = 1 - distance.cosine(theme, news_vec)
        result.append(cosine)
    new_df = pd.DataFrame(data=np.zeros([267,2]), columns=['Theme', 'Result'])
    new_df['Theme'] = theme_vectors_df['themes']
    new_df['Result'] = result

    new_df.sort_values('Result', ascending=False, inplace=True)
    return new_df
    

In [30]:
real_test = pd.read_csv("theme_contents_200_length_same.csv",index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'theme_contents_200_length_same.csv'

In [107]:
lst = []
lst2 = []
for i in tqdm(range(len(real_test))):
    a = real_test['contents'][i]
    result = test(a)
    result = result.sort_values(by='Result', ascending=False).reset_index(drop=True)
    theme = result['Theme'][0]
    cosine = result['Result'][0]
    lst.append(theme)
    lst2.append(cosine)
    

  1%|          | 401/50600 [00:06<13:22, 62.58it/s]/home/space/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 50600/50600 [13:29<00:00, 62.54it/s]


In [114]:
final_data = pd.DataFrame(list(zip(lst, lst2)), columns =['Theme', 'Similarity']) 
final_data

,Theme,Similarity
0,음식료품,0.864748
1,시스템반도체,0.819362
2,2차전지,0.835181
3,재생에너지,0.891191
4,2차전지,0.901250
...,...,...
50595,희토류,0.692710
50596,희토류,0.685137
50597,희토류,0.720064
50598,희토류,0.680904


In [115]:
our_result = pd.concat([real_test,final_data],axis=1)
our_result

,theme,contents,Theme,Similarity
0,2차전지,신종 코로나바이러스 감염증(코로나19) 사태로 인한 급락장에서 10대 그룹 시가총...,음식료품,0.864748
1,2차전지,2차전지·디스플레이 공정 장비 전문기업 나인테크가 기업인수목적회사(SPAC·스팩) ...,시스템반도체,0.819362
2,2차전지,2차전지·디스플레이 공정 장비 전문기업 나인테크가 교보7호스팩과 합병을 통한 코스...,2차전지,0.835181
3,2차전지,'울산형 뉴딜사업' 추진 등 코로나19 이후 지역경제 살리기에 나선 울산에 반가운...,재생에너지,0.891191
4,2차전지,코스닥 2차 전지 소재 기업 에코프로비엠이 압도적 기술력을 바탕으로 가파른 성장 ...,2차전지,0.901250
...,...,...,...,...
50595,희토류,희귀금속(희토류 등) 테마 주요 종목 등락률씽크풀 로보알고리즘 RASSI의 분석에...,희토류,0.692710
50596,희토류,희귀금속(희토류 등) 테마 주요 종목 등락률씽크풀 로보알고리즘 RASSI의 분석에...,희토류,0.685137
50597,희토류,희귀금속(희토류 등) 테마 주요 종목 등락률씽크풀 로보알고리즘 RASSI의 분석에...,희토류,0.720064
50598,희토류,희귀금속(희토류 등) 테마 주요 종목 등락률씽크풀 로보알고리즘 RASSI의 분석에...,희토류,0.680904


In [116]:
cleansed = our_result[our_result['Similarity']>=0.95].reset_index(drop=True)
cleansed = cleansed[['Theme','contents']]
cleansed

,Theme,contents
0,FinTech,웹사이트: www.materialise.com 출처: MaterialiseM...
1,낸드,Unlicensed frequency band to boost Wi-Fi speed...
2,낸드,A prototype wearable display that connects to ...
3,낸드,S. Korea temporarily halts war remains recover...
4,낸드,(LEAD) S. Korea temporarily halts war remains ...
...,...,...
1234,희토류,테마가 강세다. 전일 대비 2.45% 상승세이다. 유니온머티리얼(047400)+7...
1235,희토류,테마가 강세다. 전일 대비 4.79% 상승세이다. 상보(027580)+22.54%...
1236,희토류,테마가 강세다. 전일 대비 3.09% 상승세이다. 노바텍(285490)+13.32...
1237,희토류,테마가 강세다. 전일 대비 4.21% 상승세이다. 쎄노텍(222420)+20.00...


In [125]:
a = cleansed.groupby('Theme').count().sort_values(by='contents',ascending=False)
a[(a['contents']>3) & (a['contents']<10)]

,contents
Theme,
재생에너지,9
보톡스,9
나파모스타트,8
양돈,8
돼지열병,6
비메모리,5
국가보안법,5
폴더블폰,5
가스터빈,4


In [119]:
cleansed.groupby('Theme').count().sort_values(by='contents',ascending=False)

,contents
Theme,
음식료업종,274
FPCB,199
가상화폐,159
낸드,119
FinTech,83
덱사메타손,33
메트포르민,30
클로로퀸,25
언팩,25
